<a href="https://colab.research.google.com/github/JoJoWeixiChen/Data-Science-Case1/blob/main/weixi_chen_vanderbilt_edu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Grople syrup case study

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, median_absolute_error

### Load the datasets

In [ ]:
production_quanity = pd.read_csv('/Production Quantity.csv', sep = ',')
production_quanity.head()

,start_date,end_date,prod,region_id
0,2015-01-01T00:00:00.000Z,2015-01-31T00:00:00.000Z,171725,93
1,2015-02-01T00:00:00.000Z,2015-02-28T00:00:00.000Z,188325,93
2,2015-03-01T00:00:00.000Z,2015-03-31T00:00:00.000Z,247856,93
3,2015-04-01T00:00:00.000Z,2015-04-30T00:00:00.000Z,282791,93
4,2015-05-01T00:00:00.000Z,2015-05-31T00:00:00.000Z,291057,93


In [ ]:
daily_precipitation = pd.read_csv('/Daily Precipitation.csv', sep = ',')
daily_precipitation.head()

,start_date,end_date,precip,region_id
0,2014-01-01T00:00:00.000Z,2014-01-01T00:00:00.000Z,1.392393,93
1,2014-01-02T00:00:00.000Z,2014-01-02T00:00:00.000Z,0.315380,93
2,2014-01-03T00:00:00.000Z,2014-01-03T00:00:00.000Z,2.347846,93
3,2014-01-04T00:00:00.000Z,2014-01-04T00:00:00.000Z,21.466357,93
4,2014-01-05T00:00:00.000Z,2014-01-05T00:00:00.000Z,32.823651,93


In [ ]:
daily_soil_mositure = pd.read_csv('/Daily Soil Mositure.csv', sep = ',')
daily_soil_mositure.head()

,start_date,end_date,smos,region_id
0,2014-01-01T00:00:00.000Z,2014-01-01T00:00:00.000Z,0.310787,93
1,2014-01-02T00:00:00.000Z,2014-01-02T00:00:00.000Z,0.192271,93
2,2014-01-03T00:00:00.000Z,2014-01-03T00:00:00.000Z,0.265683,93
3,2014-01-04T00:00:00.000Z,2014-01-04T00:00:00.000Z,0.265683,93
4,2014-01-05T00:00:00.000Z,2014-01-05T00:00:00.000Z,0.230782,93


In [ ]:
daily_temperature = pd.read_csv('/Daily Temperature.csv', sep = ',')
daily_temperature.head()

,start_date,end_date,temp,region_id
0,2014-01-02T00:00:00.000Z,2014-01-02T00:00:00.000Z,24.707605,93
1,2014-01-11T00:00:00.000Z,2014-01-11T00:00:00.000Z,26.421176,93
2,2014-01-15T00:00:00.000Z,2014-01-15T00:00:00.000Z,24.305642,93
3,2014-01-20T00:00:00.000Z,2014-01-20T00:00:00.000Z,25.142744,93
4,2014-01-23T00:00:00.000Z,2014-01-23T00:00:00.000Z,23.653173,93


In [ ]:
eight_day_ndvi = pd.read_csv('/Eight Day NDVI.csv', sep = ',')
eight_day_ndvi.head()

,start_date,end_date,ndvi,region_id
0,2013-12-27T00:00:00.000Z,2014-01-03T00:00:00.000Z,0.679106,93
1,2014-01-01T00:00:00.000Z,2014-01-08T00:00:00.000Z,0.701431,93
2,2014-01-09T00:00:00.000Z,2014-01-16T00:00:00.000Z,0.745149,93
3,2014-01-17T00:00:00.000Z,2014-01-24T00:00:00.000Z,0.747391,93
4,2014-01-25T00:00:00.000Z,2014-02-01T00:00:00.000Z,0.768325,93


In [ ]:
predicted_production_qty = pd.read_csv('/predicted_production_qty.csv', sep = ',')
predicted_production_qty.head()

,start_date,end_date,prod,region_id
0,2021-01-01T00:00:00.000Z,2021-01-31T00:00:00.000Z,NaN,93
1,2021-02-01T00:00:00.000Z,2021-02-28T00:00:00.000Z,NaN,93
2,2021-03-01T00:00:00.000Z,2021-03-31T00:00:00.000Z,NaN,93
3,2021-04-01T00:00:00.000Z,2021-04-30T00:00:00.000Z,NaN,93
4,2021-05-01T00:00:00.000Z,2021-05-31T00:00:00.000Z,NaN,93


### Feature engineering

Considering the dataset is clean, so we do not need to do the data cleaning work. Now, we can move on to the feature engineering part.

In [ ]:
daily_precipitation['month'] = [i[:-17] for i in daily_precipitation['start_date'].tolist()]
ave_precip = daily_precipitation.groupby(['month', 'region_id']).agg({'precip':'mean'})
ave_precip.head()

precip
month   region_id          
2014-01 93         2.541758
        94         0.903706
        95         2.754678
        97         1.486648
        98         4.303181

In [ ]:
daily_soil_mositure['month'] = [i[:-17] for i in daily_soil_mositure['start_date'].tolist()]
ave_smos = daily_soil_mositure.groupby(['month', 'region_id']).agg({'smos':'mean'})
ave_smos.head()

smos
month   region_id          
2014-01 93         0.287870
        94         0.285901
        95         0.268944
        97         0.295487
        98         0.207876

In [ ]:
daily_temperature['month'] = [i[:-17] for i in daily_temperature['start_date'].tolist()]
ave_temp = daily_temperature.groupby(['month', 'region_id']).agg({'temp':'mean'})
ave_temp.head()

temp
month   region_id           
2014-01 93         25.482158
        94         25.857623
        95         22.430714
        97         25.554444
        98         23.789993

In [ ]:
eight_day_ndvi['month'] = [i[:-17] for i in eight_day_ndvi['end_date'].tolist()]
ave_ndvi = eight_day_ndvi.groupby(['month', 'region_id']).agg({'ndvi':'mean'})
ave_ndvi.head()

ndvi
month   region_id          
2014-01 93         0.718269
        94         0.767824
        95         0.696097
        97         0.756034
        98         0.793170

In [ ]:
production_quanity['month'] = [i[:-17] for i in production_quanity['end_date'].tolist()]
production_quanity.head()

,start_date,end_date,prod,region_id,month
0,2015-01-01T00:00:00.000Z,2015-01-31T00:00:00.000Z,171725,93,2015-01
1,2015-02-01T00:00:00.000Z,2015-02-28T00:00:00.000Z,188325,93,2015-02
2,2015-03-01T00:00:00.000Z,2015-03-31T00:00:00.000Z,247856,93,2015-03
3,2015-04-01T00:00:00.000Z,2015-04-30T00:00:00.000Z,282791,93,2015-04
4,2015-05-01T00:00:00.000Z,2015-05-31T00:00:00.000Z,291057,93,2015-05


## Table join and data preparation

In [ ]:
input_table = ave_precip.merge(ave_smos, on = ('month', 'region_id'), how = 'left')\
                        .merge(ave_temp, on = ('month', 'region_id'), how = 'left')\
                        .merge(ave_ndvi, on = ('month', 'region_id'), how = 'left')
input_table = input_table.fillna(axis = 0, method = 'ffill')
input_table.head()

precip      smos       temp      ndvi
month   region_id                                         
2014-01 93         2.541758  0.287870  25.482158  0.718269
        94         0.903706  0.285901  25.857623  0.767824
        95         2.754678  0.268944  22.430714  0.696097
        97         1.486648  0.295487  25.554444  0.756034
        98         4.303181  0.207876  23.789993  0.793170

In [ ]:
output_table = production_quanity[['month','region_id', 'prod']]
output_table.head()

,month,region_id,prod
0,2015-01,93,171725
1,2015-02,93,188325
2,2015-03,93,247856
3,2015-04,93,282791
4,2015-05,93,291057


In [ ]:
data = input_table.merge(output_table, on = ('month', 'region_id'), how = 'left')
data.head()

,month,region_id,precip,smos,temp,ndvi,prod
0,2014-01,93,2.541758,0.287870,25.482158,0.718269,NaN
1,2014-01,94,0.903706,0.285901,25.857623,0.767824,NaN
2,2014-01,95,2.754678,0.268944,22.430714,0.696097,NaN
3,2014-01,97,1.486648,0.295487,25.554444,0.756034,NaN
4,2014-01,98,4.303181,0.207876,23.789993,0.793170,NaN


In [ ]:
data['lag_precip'] = ['NaN' for i in range(len(data['precip']))]
data['lag_precip'][2:] = data['precip'][:-2]
data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,month,region_id,precip,smos,temp,ndvi,prod,lag_precip,lag_smos
0,2014-01,93,2.541758,0.287870,25.482158,0.718269,NaN,NaN,NaN
1,2014-01,94,0.903706,0.285901,25.857623,0.767824,NaN,NaN,NaN
2,2014-01,95,2.754678,0.268944,22.430714,0.696097,NaN,2.541758,0.28787
3,2014-01,97,1.486648,0.295487,25.554444,0.756034,NaN,0.903706,0.285901
4,2014-01,98,4.303181,0.207876,23.789993,0.793170,NaN,2.754678,0.268944


In [ ]:
data['lag_smos'] = ['NaN' for i in range(len(data['smos']))]
data['lag_smos'][2:] = data['smos'][:-2]
data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,month,region_id,precip,smos,temp,ndvi,prod,lag_precip,lag_smos
0,2014-01,93,2.541758,0.287870,25.482158,0.718269,NaN,NaN,NaN
1,2014-01,94,0.903706,0.285901,25.857623,0.767824,NaN,NaN,NaN
2,2014-01,95,2.754678,0.268944,22.430714,0.696097,NaN,2.541758,0.28787
3,2014-01,97,1.486648,0.295487,25.554444,0.756034,NaN,0.903706,0.285901
4,2014-01,98,4.303181,0.207876,23.789993,0.793170,NaN,2.754678,0.268944


In [ ]:
data['lag_temp'] = ['NaN' for i in range(len(data['temp']))]
data['lag_temp'][1:] = data['temp'][:-1]
data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,month,region_id,precip,smos,temp,ndvi,prod,lag_precip,lag_smos,lag_temp
0,2014-01,93,2.541758,0.287870,25.482158,0.718269,NaN,NaN,NaN,NaN
1,2014-01,94,0.903706,0.285901,25.857623,0.767824,NaN,NaN,NaN,25.482158
2,2014-01,95,2.754678,0.268944,22.430714,0.696097,NaN,2.541758,0.28787,25.857623
3,2014-01,97,1.486648,0.295487,25.554444,0.756034,NaN,0.903706,0.285901,22.430714
4,2014-01,98,4.303181,0.207876,23.789993,0.793170,NaN,2.754678,0.268944,25.554444


## Modeling

As the question mentioned, 'It takes a few months for the fruits to grow on the trees which bear them. It also takes a few days to extract the syrup from the fruits after they have been harvested.', so it is better to use lag of some attributes as input.

In [ ]:
# linear model seems to be a good choice for this situation
y = data.iloc[120:839,6]
x = data.iloc[120:839,[5,7,8,9]]

In [ ]:
modelRegL = LinearRegression()
modelRegL.fit(x, y)
fit1 = modelRegL.predict(x)

In [ ]:
R2 = modelRegL.score(x, y)
print(R2)

0.29320017105793106


In [ ]:
x1 = data.iloc[840:960,[5,7,8,9]]

In [ ]:
y1 = modelRegL.predict(x1)

In [ ]:
predicted_production_qty['prod'] = y1

In [ ]:
predicted_production_qty.to_csv("/predicted_production_qty.csv", index = False)